In [50]:
import pandas as pd
import numpy as np
import pickle as pkl

In [ ]:
data16_17 = pd.read_csv('../../../Data/07_FinalProject/Most-Recent-Cohorts-All-Data-Elements.csv')

In [55]:
def normalizeColumn(column, data):
    if isinstance(data[column][0], numbers.Number):
        return [float(i)/sum(data[column].values) for i in data[column].values]
    else:
        return data[column]

def normalizeData(data):
    for column in data.columns:
        data[column] = normalizeColumn(column, data)
    return data

def cleanColumn(column, data):
    if type(data[column][0]) == type('.'):
        data[column][data[column] == 'PrivacySuppressed'] = np.nan
    return data[column]

def cleanData(data):
    for column in data.columns:
        data[column] = cleanColumn(column, data)
    return data

In [56]:
def imputeValue(column, data):
    column_data = pd.to_numeric(data[column])
    column_mean = column_data.mean()
    for i, row in enumerate(column_data):
        if np.isnan(row):
            column_data.iloc[i] = column_mean
    return column_data

def imputeNumeric(columns, data):
    for column in columns:
        data[column] = imputeValue(column, data)
    return data

def processCategorical(columns, data):
    for column in columns:
        data[column] = data[column].astype('category')
    return data

In [57]:
data = pd.read_csv('./ROI.csv')

In [58]:
data = cleanData(data)

/anaconda3/envs/NLP/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [59]:
target_C100 = []

C100_4 = data['C100_4']
C100_L4 = data['C100_L4']

data = data.drop(['C100_4', 'C100_L4'], 1)

for i in range(len(C100_4)):
    if not np.isnan(C100_4[i]):
        if not np.isnan(C100_L4[i]):
            raise Exception('No school is like this')
        target_C100.append(C100_4[i])
    else:
        target_C100.append(C100_L4[i])
        
data['C100_4'] = target_C100

In [60]:
numeric_columns = ['COMPL_RPY_1YR_RT', 'COMPL_RPY_3YR_RT', 'COMPL_RPY_5YR_RT', 
                   'COMPL_RPY_7YR_RT', 'DEBT_MDN_SUPP', 'GRAD_DEBT_MDN', 'DEBT_N', 
                   'GRAD_DEBT_N', 'LATITUDE', 'LONGITUDE', 'NUMBRANCH']

categorical_column = ['REGION', 'LOCALE', 'SCH_DEG', 'MAIN', 'HIGHDEG', 'CONTROL', 'ST_FIPS']

Y_column = ['earning', 'cost']

data = imputeNumeric(numeric_columns, data)
data = processCategorical(categorical_column, data)

In [61]:
test_column = []
test_column.extend(numeric_columns)
test_column.extend(categorical_column)
test_column.extend(Y_column)
data_test = data[test_column]

In [62]:
pkl.dump(data_test, open("data_test.p", "wb"))

In [63]:
data_test.isna().sum()

COMPL_RPY_1YR_RT       0
COMPL_RPY_3YR_RT       0
COMPL_RPY_5YR_RT       0
COMPL_RPY_7YR_RT       0
DEBT_MDN_SUPP          0
GRAD_DEBT_MDN          0
DEBT_N                 0
GRAD_DEBT_N            0
LATITUDE               0
LONGITUDE              0
NUMBRANCH              0
REGION                 0
LOCALE               438
SCH_DEG              308
MAIN                   0
HIGHDEG                0
CONTROL                0
ST_FIPS                0
earning             1646
cost                1203
dtype: int64